<a href="https://colab.research.google.com/github/faithrts/Science_Explainers/blob/main/dataset/science_explainer_dataset_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
### importing libraries

# basic libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# libraries for web scraping
from bs4 import BeautifulSoup
import requests
import re
import codecs

# to download files
from google.colab import files

# Importing article URLs

In [2]:
### cloning git repo and saving csv file of URLs into dataframe

!git clone https://github.com/faithrts/Science_Explainers

Cloning into 'Science_Explainers'...
remote: Enumerating objects: 1882, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 1882 (delta 54), reused 36 (delta 19), pack-reused 1791
Receiving objects: 100% (1882/1882), 97.77 MiB | 9.92 MiB/s, done.
Resolving deltas: 100% (842/842), done.
Updating files: 100% (18/18), done.


In [3]:
### unzips txt files

!unzip Science_Explainers/dataset/science_txt_files.zip

Archive:  Science_Explainers/dataset/science_txt_files.zip
   creating: science_txt_files/
   creating: science_txt_files/CBC/
  inflating: science_txt_files/CBC/NewStudyProposesToUncoverWhere.txt  
  inflating: science_txt_files/CBC/StudentsSayMisinformationAboundsOnlineExperts.txt  
  inflating: science_txt_files/CBC/AlaskapoxDeathBringsAttentionToSmall.txt  
  inflating: science_txt_files/CBC/AlbertasOilsandsPumpOutMorePollutants.txt  
  inflating: science_txt_files/CBC/ScientistsCanDetectAnimalSpeciesBy.txt  
  inflating: science_txt_files/CBC/FungiMayNotZombifyYouBut.txt  
  inflating: science_txt_files/CBC/NewDiscoveriesShowEarlyHumansLived.txt  
  inflating: science_txt_files/CBC/SnowDayNowEclipseDayWhy.txt  
  inflating: science_txt_files/CBC/GulfOfStLawrenceRedfishPopulation.txt  
  inflating: science_txt_files/CBC/EuropesJuiceMissionWillGetUs.txt  
  inflating: science_txt_files/CBC/EasternOntarioFarmerDiscoversNewVariety.txt  
  inflating: science_txt_files/CBC/Environmental

In [9]:
### saving csv file of URLs into dataframe

urls_df = pd.read_csv('Science_Explainers/dataset/article_urls.csv')
# urls_df = pd.read_csv('article_urls.csv')

# replaces all NaN instances with 0
urls_df = urls_df.fillna('')

# Webscraping helper functions

In [10]:
header_ex = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

In [11]:
def get_soup(url, header = ''):
  if header == '':
    header = header_ex

  response = requests.get(url, headers = header)

  soup = BeautifulSoup(response.text, 'html.parser')
  return soup

In [12]:
def create_txt_file(soup, filename):

  source = re.search('(.*)(?=\/)', filename)[0]

  find_all = {'ATLANTIC': 'soup.findAll("p")',
              'CBC': 'soup.find("div", {"class": "story"}).findAll("p")',
              'CNN': 'soup.findAll("p")',
              'GLOBE_AND_MAIL': 'soup.findAll("p")',
              'MASSIVE_SCI': 'soup.find("div", {"class": "bodytext"}).findAll("p")',
              'NATIONAL_GEOGRAPHIC': 'soup.findAll("p")',
              'NATIONAL_OBSERVER': 'soup.findAll("p")',
              'NPR': 'soup.find("div", {"class": "image_data"}).find_all_next("p", {"class": None})',
              'REUTERS': 'soup.find("div", {"class": "article-body__content__17Yit"}).findAll("p")'}

  dont_keep = ['read transcribed audio', '\neditor\'s note','sign up for', 'this article was published', 'this story was originally published', 'sponsor']

  # VERY weird egde case
  if source == 'NPR':
    try:
      passages = eval(find_all[source])
    except:
      passages = soup.find("span", {"class": "credit"}).find_all_next("p", {"class": None})

  else:
    # looks for the text of the story
    passages = eval(find_all[source])

  # if, for some reason, there's no text at this url
  if len(passages) == 0:
    return False

  # else, creates a new file
  cur_file = open('science_txt_files/' + filename, 'w+')

  first_line = True

  # iterates through each passage in the article by finding <p> tags
  for passage in passages:

    # extracts the text
    text = passage.get_text()

    # fixing spacing
    text = text.replace(u'\xa0', u' ')
    text = text.replace(u'  ', u' ')

    if any(substring in text.lower() for substring in dont_keep):
      continue
    if source == 'REUTERS' and first_line:
      text = re.sub('.*(?<=\(Reuters\) - )', '', text)
      first_line = False

    # adding a newline before the next passage
    text += '\n'

    # writing the text to the current file
    cur_file.write(text)

  cur_file.close()

  return True

In [13]:
def create_txt_file_reuters(soup, filename, verbose = False):

  cur_file = open('science_txt_files/' + filename, 'w+')
  all_text = ''

  first = True

  for result in soup.findAll('div', {'data-testid': not None}):

    cur_text = result.text

    if cur_text == '':
      continue

    if first:
      remove_reuters = re.search('(?<=\(Reuters\) - ).*', cur_text)

      if remove_reuters != None:
        cur_text = remove_reuters[0]

      first = False

      all_text += cur_text

    elif 'Advertisement · Scroll'not in cur_text and 'Purchase Licensing Rights' not in cur_text:
      all_text += '\n' + cur_text

  if verbose:
    print(all_text)

  cur_file.write(all_text)
  cur_file.close()

In [14]:
def source_finder(url):
  # testing default source finder
  source = re.search('(?<=https:\/\/www\.)(.*?)(=?\.)', url).group(1)

  # if no source found, tests another link format
  if len(source) == 0:
    source = re.search('(?<=https:\/\/)(.*?)(=?\.)', url).group(1)

  return source

In [15]:
def title_cleaner(title):
  # removes punctuation
  title = re.sub(r'[^\w\s]', '', title)
  return ''.join(title.title().split()[:6])

In [16]:
def title_finder(soup, source):
  # testing default title finder
  title = soup.findAll('h1')

  # if the title is empty, test another title finder
  if len(title) == 0:
    # testing another title finder
    title = soup.findAll('title')

  title = title[0].get_text()
  title = title.replace('\n', '')

  # if the title is not empty, return it
  if len(title) != 0:
    return title

  # another title finder format for CNN articles
  if source == 'cnn':
    title = soup.findAll('h1', {'class': 'headline__text inline-placeholder'})[0].get_text()
    title = title.replace('  ', '')
    title = title.replace('\n', '')

    return title

In [17]:
def add_text_column(df):
  # adding a column for the text contents
  df['TEXT'] = ''

  for index, row in df.iterrows():
    filename = row['FILENAME']
    text = codecs.open('txt_files/' + filename, "r", encoding='utf8').read()

    df.at[index, 'TEXT'] = text

  return df

In [18]:
def update_dataset(urls_df, dataset):

  # the urls currently in the dataset
  preexisting_urls = dataset['URL'].tolist()

  new_dataset = dataset.copy()

  # the urls that are added to the new dataset
  new_urls = []

  # iterating through each column of the df, which translates to each source
  # of science explainers
  for source in urls_df.columns:

    source = source.replace(' ', '_')

    print(source)

    # iterating through the rows of the current column of the df, which
    # translates to the article urls from the current source
    for index, row in urls_df[source].items():

      url = row

      # skips empty urls and urls already in the dataset
      if url == '' or url in preexisting_urls:
        continue

      # if the url isn't already in the dataset, creates a new entry in the dataset
      new_urls.append(url)

      # gets the website content
      r = requests.get(url)
      soup = BeautifulSoup(r.content, 'html.parser')

      # extracts the title of the article
      title = title_finder(soup, source)

      # edit the title for the filename (title case and only the first 5 words)
      title_cut = title_cleaner(title)

      # creates a new file, writes entire article to it, then saves in the txt_files folder
      filename = source.replace(' ', '_') + '/' + title_cut + '.txt'
      create_txt_file(soup, filename)

      print('\t' + filename)

      # retrieves the date of publication
      date = all_date_finder(source, soup)

      # adds a row to the science explainer dataset with the info of this article
      new_row = pd.DataFrame({'FILENAME': filename,
                              'TITLE': title,
                              'SOURCE': source,
                              'DATE PUBLISHED': date,
                              'URL': url},
                             index = [0])
      #dataset = dataset.append(new_row, ignore_index=True)
      new_dataset = pd.concat([new_dataset, new_row], ignore_index = True)

  # returns the updated dataset along with the urls added to it
  return new_dataset, new_urls

In [19]:
def update_dataset_reuters(html_file, url, title, date, df):

  title_cut = title_cleaner(title)

  # creates a new file, writes entire article to it, then saves in the txt_files folder
  filename = 'REUTERS/' + title_cut + '.txt'

  print('\n' + filename + '\n')

  with open(html_file) as temp_file:
    lines = temp_file.readlines()
    lines_as_str = '\n'.join(lines)

    soup = BeautifulSoup(lines_as_str, "html.parser")

  create_txt_file_reuters(soup, filename, True)

  # if this is already an entry
  if filename in df['FILENAME'].tolist():
    return df

  # adds a row to the science explainer dataset with the info of this article
  new_row = pd.DataFrame({'FILENAME': filename,
                              'TITLE': title,
                              'SOURCE': 'REUTERS',
                              'DATE PUBLISHED': date,
                              'URL': url},
                             index = [0])


  new_dataset = pd.concat([df, new_row], ignore_index = True)

  # returns the updated dataset along with the urls added to it
  return new_dataset

## Date finder functions

In [20]:
def basic_date_finder(soup):
  date_bunch = soup.select_one('time')
  date = re.search('(?<= datetime=")(.*?)(=?T)', str(date_bunch)).group(1)
  return date

In [21]:
def atlantic_date_finder(soup):
  return basic_date_finder(soup)

In [22]:
def cbc_date_finder(soup):
  return basic_date_finder(soup)

In [23]:
def cnn_date_finder(soup):
  date_bunch = soup.find('link', {'rel': 'canonical'})
  date = re.search('(?<=cnn\.com\/)(\d\d\d\d\/\d\d\/\d\d)(=?)', str(date_bunch))

  # edge case
  if date == None:
    soup_as_string = str(soup)
    date = re.search('(?<=publish_date: \\\')(.*?)(=?\\\')', soup_as_string)

  date = date.group(1)
  date = date.replace('/', '-')

  return date

In [24]:
def globe_and_mail_date_finder(soup):
  return basic_date_finder(soup)

In [25]:
def massive_sci_date_finder(soup):
  return basic_date_finder(soup)

In [26]:
def national_geographic_date_finder(soup):
  soup_as_string = str(soup)
  date = re.search('(?<="pbDt":")(.*?)(=?T)', soup_as_string)

  # edge case
  if date == None:
    soup_as_string = str(soup)
    date = re.search('(?<=datePublished":")(.*?)(=?T)', soup_as_string)

  date = date.group(1)

  return date

In [27]:
def national_observer_date_finder(soup):
  soup_as_string = str(soup)
  date = re.search('(?<="datePublished": ")(.*?)(=?T)', soup_as_string).group(1)
  return date

In [28]:
def npr_date_finder(soup):
  date_bunch = soup.find('link', {'rel': 'canonical'})
  date = re.search('(\d\d\d\d\/\d\d\/\d\d)', str(date_bunch))

  # edge case
  if date == None:
    soup_as_string = str(soup)
    date = re.search('(?<=datePublished":")(.*?)(=?T)', soup_as_string)

  date = date.group(1)
  date = date.replace('/', '-')

  return date

In [5]:
def reuters_date_finder(soup):
  date_bunch = soup.find('link', {'rel': 'canonical'})
  date = re.search('(\d\d\d\d-\d\d-\d\d)', str(date_bunch)).group(1)
  return date

In [6]:
def all_date_finder(source_name, soup):
  # the name of the specific source's date finder function name
  finder_func = source_name.lower() + '_date_finder'

  # calls the specific source's date finder function
  return (eval(finder_func)(soup))

# Testing

In [7]:
### the dataset before adding new data

science_explainers_df = pd.read_csv('Science_Explainers/dataset/science_explainers_dataset.csv')

In [29]:
### making a copy of urls_df without the REUTERS column (due to paywall issues)

urls_no_reuters_df = urls_df.iloc[:, 0:-1]

In [30]:
### updating the dataset
science_explainers_df, new_urls = update_dataset(urls_no_reuters_df, science_explainers_df)

ATLANTIC
CBC
CNN
GLOBE_AND_MAIL
MASSIVE_SCI
NATIONAL_GEOGRAPHIC
NATIONAL_OBSERVER
NPR


In [31]:
### the added files
new_urls

[]

# Adding Reuters articles manually

In [ ]:
### unzip extra txt files of html

!unzip extra_content.zip

Archive:  extra_content.zip
   creating: extra_content/
  inflating: extra_content/reuters_21.txt  
  inflating: __MACOSX/extra_content/._reuters_21.txt  
  inflating: extra_content/reuters_20.txt  
  inflating: __MACOSX/extra_content/._reuters_20.txt  
  inflating: extra_content/reuters_22.txt  
  inflating: __MACOSX/extra_content/._reuters_22.txt  
  inflating: extra_content/reuters_23.txt  
  inflating: __MACOSX/extra_content/._reuters_23.txt  
  inflating: extra_content/reuters_27.txt  
  inflating: __MACOSX/extra_content/._reuters_27.txt  
  inflating: extra_content/reuters_26.txt  
  inflating: __MACOSX/extra_content/._reuters_26.txt  
  inflating: extra_content/reuters_24.txt  
  inflating: __MACOSX/extra_content/._reuters_24.txt  
  inflating: extra_content/reuters_30.txt  
  inflating: __MACOSX/extra_content/._reuters_30.txt  
  inflating: extra_content/reuters_18.txt  
  inflating: __MACOSX/extra_content/._reuters_18.txt  
  inflating: extra_content/reuters_19.txt  
  inflati

In [ ]:
### iterating through each additional reuters URL

last_30 = urls_df['REUTERS'].tolist()[-30:]

for i in range(1, 31):
  cur_url = last_30[i - 1]

  title = ' '.join(re.search('(.*?)(?=\d\d\d\d-\d\d-\d\d)', cur_url)[0].split('/')[-1].split('-')).strip(' ').title()
  date = re.search('\d\d\d\d-\d\d-\d\d', cur_url)[0]

  txt_title = f'extra_content/reuters_{i}.txt'

  science_explainers_df = update_dataset_reuters(txt_title, cur_url, title, date, science_explainers_df)


REUTERS/PeopleWithTailsNoBecauseThis.txt

Director James Cameron's "Avatar" movies are populated by a species of outsized blue beings resembling humans, except with tails. So why does our species lack a tail, considering that our evolutionary forerunners in the primate lineage had them?
Scientists on Wednesday identified what might be the genetic mechanism behind the tailless condition of us and our ape ancestors - a mutation in a gene instrumental in embryonic development. The tail was a feature of most vertebrates for more than half a billion years, and its loss may have offered advantages as our ancestors moved from the trees to the ground, they said.
The researchers compared the DNA of two groups of primates: monkeys, which have tails, and hominoids - humans and apes - which do not. They found a mutation in a gene called TBXT that was present in people and apes but absent in monkeys. To test the effects of this mutation, the researchers genetically modified laboratory mice to have

In [ ]:
science_explainers_df

,FILENAME,TITLE,SOURCE,DATE PUBLISHED,URL
0,ATLANTIC/15DegreesWasNeverTheEnd.txt,1.5 Degrees Was Never the End of the World,ATLANTIC,2023-02-01,https://www.theatlantic.com/science/archive/20...
1,ATLANTIC/ABasicPremiseOfAnimalConservation.txt,A Basic Premise of Animal Conservation Looks S...,ATLANTIC,2023-02-22,https://www.theatlantic.com/science/archive/20...
2,ATLANTIC/AClearIndicationThatClimateChange.txt,A Clear Indication That Climate Change Is Burn...,ATLANTIC,2023-06-12,https://www.theatlantic.com/science/archive/20...
3,ATLANTIC/ACognitiveRevolutionInAnimalResearch.txt,A Cognitive Revolution in Animal Research,ATLANTIC,2023-03-19,https://www.theatlantic.com/science/archive/20...
4,ATLANTIC/ACrucialBarrierAgainstHurricanesIs.txt,A Crucial Barrier Against Hurricanes Is at Risk,ATLANTIC,2023-10-01,https://www.theatlantic.com/science/archive/20...
...,...,...,...,...,...
1165,REUTERS/WereReleasingLifeHeatWaveSpeeds.txt,Were Releasing Life Heat Wave Speeds Turtle Ha...,REUTERS,2023-11-16,https://www.reuters.com/world/americas/were-re...
1166,REUTERS/BoneBiteMarksRevealDinosaurPredator.txt,Bone Bite Marks Reveal Dinosaur Predator Prey ...,REUTERS,2023-11-16,https://www.reuters.com/science/bone-bite-mark...
1167,REUTERS/ScientistsIdentifyNewDinosaurSpeciesFo...,Scientists Identify New Dinosaur Species Footp...,REUTERS,2023-11-23,https://www.reuters.com/science/scientists-ide...
1168,REUTERS/ScientistsDiscernInternalStructureMyst...,Scientists Discern Internal Structure Mysterio...,REUTERS,2023-11-15,https://www.reuters.com/science/scientists-dis...


# Downloading files

In [35]:
### resorts df to group entries together based on source

science_explainers_df = science_explainers_df.sort_values(['SOURCE','FILENAME']).reset_index(drop = True)

In [36]:
### downloading science explainers dataset

science_explainers_df.to_csv('science_explainers_dataset.csv', index = False)
files.download('science_explainers_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
urls_df.to_csv('article_urls.csv', index = False)
files.download('article_urls.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
### downloading txt files

!zip -r science_txt_files.zip science_txt_files
files.download("science_txt_files.zip")

  adding: science_txt_files/ (stored 0%)
  adding: science_txt_files/CBC/ (stored 0%)
  adding: science_txt_files/CBC/NewStudyProposesToUncoverWhere.txt (deflated 53%)
  adding: science_txt_files/CBC/StudentsSayMisinformationAboundsOnlineExperts.txt (deflated 55%)
  adding: science_txt_files/CBC/AlaskapoxDeathBringsAttentionToSmall.txt (deflated 53%)
  adding: science_txt_files/CBC/AlbertasOilsandsPumpOutMorePollutants.txt (deflated 56%)
  adding: science_txt_files/CBC/ScientistsCanDetectAnimalSpeciesBy.txt (deflated 56%)
  adding: science_txt_files/CBC/FungiMayNotZombifyYouBut.txt (deflated 55%)
  adding: science_txt_files/CBC/NewDiscoveriesShowEarlyHumansLived.txt (deflated 55%)
  adding: science_txt_files/CBC/SnowDayNowEclipseDayWhy.txt (deflated 55%)
  adding: science_txt_files/CBC/GulfOfStLawrenceRedfishPopulation.txt (deflated 52%)
  adding: science_txt_files/CBC/EuropesJuiceMissionWillGetUs.txt (deflated 52%)
  adding: science_txt_files/CBC/EasternOntarioFarmerDiscoversNewVariet

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>